In [ ]:
import os
import shutil
import random

dataset_folder = '102flowers/dataset'
train_folder= '102flowers/trainfolder'
validation_folder = '102flowers/validationfolder'
test_folder = '102flowers/testfolder'

os.makedirs(dataset_folder,exist_ok=True)
os.makedirs(train_folder,exist_ok=True)
os.makedirs(validation_folder,exist_ok=True)
os.makedirs(test_folder,exist_ok=True)

train_ratio=0.7
validation_ratio = 0.15
test_ratio = 0.15

image_files = os.listdir(dataset_folder)
random.shuffle(image_files)

num_images = len(image_files)
train_split= int(num_images*train_ratio)
validation_split= train_split + int(num_images * validation_ratio)

for i,image_file in enumerate(image_files):
    src_path= os.path.join(dataset_folder,image_files)
    if i < train_split:
        dst_path= os.path.join(train_folder,image_files)
    elif i < validation_split:
        dst_path = os.path.join(validation_folder,image_files)
    else:
        dst_path = os.path.join(test_folder,image_files)
    shutil.copy(src_path,dst_path)

print('Dataset are seprated successfully')    
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tarfile

dataset_path = '102flowers.tgz'

with tarfile.open(dataset_path, 'r:gz') as tar:
    tar.extractall()
    
    
input_shape=(224,224,3)

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range= 0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '102flowers',
    target_size=input_shape[:2],
    batch_size=128,
    class_mode='categorical',
    subset='training'
)
validation_generator=train_datagen.flow_from_directory(
    '102flowers',
    target_size=input_shape[:2],
    batch_size=128,
    class_mode='categorical',
    subset='validation'
)

base_model=VGG16(weights='imagenet',include_top=False,input_shape=input_shape)

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
predictions=Dense(train_generator.num_classes,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer= Adam(),loss='categorical_crossentropy',metrics=['accuracy'])



history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)    
    
            
        